# 1. Setup Development Environment

In [3]:
!pip install -q -U sagemaker
!pip install -q -U transformers
!pip install -q -U "datasets[s3]"
!pip install -q -U "huggingface_hub[cli]"

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.13.1 requires botocore<1.34.132,>=1.34.70, but you have botocore 1.34.144 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.33.26 requires botocore==1.34.144, but you have botocore 1.34.131 which is incompatible.
boto3 1.34.144 requires botocore<1.35.0,>=1.34.144, but you have botocore 1.34.131 which is incompatible.


In [4]:
!pip show datasets
!pip show pandas

Name: datasets
Version: 2.20.0
Summary: HuggingFace community-driven open-source library of datasets
Home-page: https://github.com/huggingface/datasets
Author: HuggingFace Inc.
Author-email: thomas@huggingface.co
License: Apache 2.0
Location: /Users/dongkyl/.pyenv/versions/3.10.14/lib/python3.10/site-packages
Requires: aiohttp, dill, filelock, fsspec, huggingface-hub, multiprocess, numpy, packaging, pandas, pyarrow, pyarrow-hotfix, pyyaml, requests, tqdm, xxhash
Required-by: fastbook, trl
Name: pandas
Version: 2.2.2
Summary: Powerful data structures for data analysis, time series, and statistics
Home-page: https://pandas.pydata.org
Author: 
Author-email: The Pandas Development Team <pandas-dev@python.org>
License: BSD 3-Clause License
        
        Copyright (c) 2008-2011, AQR Capital Management, LLC, Lambda Foundry, Inc. and PyData Development Team
        All rights reserved.
        
        Copyright (c) 2011-2023, Open source contributors.
        
        Redistribution and us

In [6]:
# !huggingface-cli login --token YOUR_TOKEN

In [5]:
from huggingface_hub import notebook_login
notebook_login()

In [7]:
import sagemaker
import boto3
sess = sagemaker.Session()
sagemaker_session_bucket = None
if sagemaker_session_bucket is None and sess is not None:
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName="AmazonSageMaker-ExecutionRole-20230112T181165")['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

Couldn't call 'get_role' to get Role ARN from role name dongkyl to get Role path.


sagemaker role arn: arn:aws:iam::395271362395:role/service-role/AmazonSageMaker-ExecutionRole-20230112T181165
sagemaker bucket: sagemaker-us-east-1-395271362395
sagemaker session region: us-east-1


# 2. Create and prepare the dataset

[sql-crete-context](https://huggingface.co/datasets/b-mc2/sql-create-context) dataset

In [8]:
from datasets import load_dataset

dataset = load_dataset("b-mc2/sql-create-context", split="train")

In [9]:
df = dataset.to_pandas()
df.head(10)

,answer,question,context
0,SELECT COUNT(*) FROM head WHERE age > 56,How many heads of the departments are older th...,CREATE TABLE head (age INTEGER)
1,"SELECT name, born_state, age FROM head ORDER B...","List the name, born state and age of the heads...","CREATE TABLE head (name VARCHAR, born_state VA..."
2,"SELECT creation, name, budget_in_billions FROM...","List the creation year, name and budget of eac...","CREATE TABLE department (creation VARCHAR, nam..."
3,"SELECT MAX(budget_in_billions), MIN(budget_in_...",What are the maximum and minimum budget of the...,CREATE TABLE department (budget_in_billions IN...
4,SELECT AVG(num_employees) FROM department WHER...,What is the average number of employees of the...,CREATE TABLE department (num_employees INTEGER...
5,SELECT name FROM head WHERE born_state <> 'Cal...,What are the names of the heads who are born o...,"CREATE TABLE head (name VARCHAR, born_state VA..."
6,SELECT DISTINCT T1.creation FROM department AS...,What are the distinct creation years of the de...,"CREATE TABLE department (creation VARCHAR, dep..."
7,SELECT born_state FROM head GROUP BY born_stat...,What are the names of the states where at leas...,CREATE TABLE head (born_state VARCHAR)
8,SELECT creation FROM department GROUP BY creat...,In which year were most departments established?,CREATE TABLE department (creation VARCHAR)
9,"SELECT T1.name, T1.num_employees FROM departme...",Show the name and number of employees for the ...,CREATE TABLE management (department_id VARCHAR...


In [10]:
system_message = """
You are an text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA.
SCHEMA:
{schema}""".strip()


def create_conversation(sample):
    return {
        "messages": [
            {"role": "system", "content": system_message.format(schema=sample["context"])},
            {"role": "user", "content": sample["question"]},
            {"role": "assistant", "content": sample["answer"]}
        ]
    }

In [11]:
dataset = dataset.shuffle(seed=1234).select(range(12500))

dataset = dataset.map(create_conversation, remove_columns=dataset.features, batched=False)
dataset = dataset.train_test_split(test_size=2500/12500)

print(dataset["train"])
print(dataset["test"])

Map:   0%|          | 0/12500 [00:00<?, ? examples/s]

Dataset({
    features: ['messages'],
    num_rows: 10000
})
Dataset({
    features: ['messages'],
    num_rows: 2500
})


In [12]:
print(dataset["train"][345]["messages"])

[{'content': 'You are an text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA.\nSCHEMA:\nCREATE TABLE table_name_48 (title VARCHAR, studio VARCHAR)', 'role': 'system'}, {'content': 'What is Title, when Studio is "Embassy Pictures"?', 'role': 'user'}, {'content': 'SELECT title FROM table_name_48 WHERE studio = "embassy pictures"', 'role': 'assistant'}]


In [13]:
# save train_dataset to s3 using our SageMaker session
training_input_path = f's3://{sess.default_bucket()}/datasets/text-to-sql'

# save datasets to s3
dataset["train"].to_json(f"{training_input_path}/train_dataset.json", orient="records")
dataset["test"].to_json(f"{training_input_path}/test_dataset.json", orient="records")

print(f"Training data uploaded to:")
print(f"{training_input_path}/train_dataset.json")
print(f"https://s3.console.aws.amazon.com/s3/buckets/{sess.default_bucket()}/?region={sess.boto_region_name}&prefix={training_input_path.split('/', 3)[-1]}/")

Creating json from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Training data uploaded to:
s3://sagemaker-us-east-1-395271362395/datasets/text-to-sql/train_dataset.json
https://s3.console.aws.amazon.com/s3/buckets/sagemaker-us-east-1-395271362395/?region=us-east-1&prefix=datasets/text-to-sql/


# 3. Fine-Tune Gemma2 with QLoRA on Amazon Sagemaker

In [14]:
from dataclasses import dataclass, field, fields, asdict
from transformers import TrainingArguments

@dataclass
class PartialTrainingArguments:
    num_train_epochs: int
    per_device_train_batch_size: int
    gradient_accumulation_steps: int
    gradient_checkpointing: bool
    optim: str
    logging_steps: int
    save_strategy: int
    learning_rate: float
    bf16: bool
    tf32: bool
    max_grad_norm: float
    warmup_ratio: float
    lr_scheduler_type: str
    report_to: str
    output_dir: str


@dataclass
class Hyperparameters(PartialTrainingArguments):
    # path where sagemaker will save training dataset
    dataset_path: str
    model_id: str
    max_seq_len: int
    use_qlora: bool
    merge_adapters: bool

    def to_dict(self):
        return asdict(self)


# Validate training arguments
training_args_fields = {field.name for field in fields(TrainingArguments)}
partial_training_args_fields = {field.name for field in fields(PartialTrainingArguments)}
is_subset = partial_training_args_fields.issubset(training_args_fields)
assert is_subset, "All fields in PartialTrainingArguments should be in TrainingArguments"

In [15]:
hyperparameters = Hyperparameters(
    ### SagemakerArguments ###
    dataset_path="/opt/ml/input/data/training/train_dataset.json",
    model_id="google/gemma-2-9b",
    max_seq_len=1024,
    use_qlora=True,
    merge_adapters=True,
    ### TrainingArguments ###
    num_train_epochs=3,  # number of training epochs
    per_device_train_batch_size=1,  # batch size per device during training
    gradient_accumulation_steps=4,  # number of steps before performing a backward/update pass
    gradient_checkpointing=True,  # use gradient checkpointing to save memory
    optim="paged_adamw_8bit",  # use paged_adamw_8bit optimizer
    logging_steps=10,  # log every 10 steps
    save_strategy="epoch",  # save checkpoint every epoch
    learning_rate=2e-4,  # learning rate, based on QLoRA paper
    bf16=True,  # use bfloat16 precision
    tf32=True,  # use tf32 precision
    max_grad_norm=0.3,  # max gradient norm based on QLoRA paper
    warmup_ratio=0.03,  # warmup ratio based on QLoRA paper
    lr_scheduler_type="constant",  # use constant learning rate scheduler
    report_to="tensorboard",  # report metrics to tensorboard
    output_dir="/tmp/tun",  # Temporary output directory for model checkpoints
).to_dict()
print(hyperparameters)

{'num_train_epochs': 3, 'per_device_train_batch_size': 1, 'gradient_accumulation_steps': 4, 'gradient_checkpointing': True, 'optim': 'paged_adamw_8bit', 'logging_steps': 10, 'save_strategy': 'epoch', 'learning_rate': 0.0002, 'bf16': True, 'tf32': True, 'max_grad_norm': 0.3, 'warmup_ratio': 0.03, 'lr_scheduler_type': 'constant', 'report_to': 'tensorboard', 'output_dir': '/tmp/tun', 'dataset_path': '/opt/ml/input/data/training/train_dataset.json', 'model_id': 'google/gemma-2-9b', 'max_seq_len': 1024, 'use_qlora': True, 'merge_adapters': True}


In [16]:
from sagemaker.huggingface import HuggingFace

# define Training Job Name 
job_name = f'gemma-9b-it-text-to-sql'

# create the Estimator
huggingface_estimator = HuggingFace(
    entry_point          = 'run_sft.py',      # train script
    source_dir           = './scripts',       # directory which includes all the files needed for training
    instance_type        = 'ml.g5.16xlarge',  # instances type used for the training job
    instance_count       = 1,                 # the number of instances used for training
    max_run              = 2*24*3600,         # maximum runtime in seconds (days * hours * minutes * seconds)
    base_job_name        = job_name,          # the name of the training job
    role                 = role,              # Iam role used in training job to access AWS ressources, e.g. S3
    volume_size          = 300,               # the size of the EBS volume in GB
    transformers_version = '4.36',            # the transformers version used in the training job
    pytorch_version      = '2.1',             # the pytorch_version version used in the training job
    py_version           = 'py310',           # the python version used in the training job
    hyperparameters      =  hyperparameters,  # the hyperparameters passed to the training job
    disable_output_compression = True,        # not compress output to save training time and cost
    environment          = {
        "HUGGINGFACE_HUB_CACHE": "/tmp/.cache", # set env variable to cache models in /tmp
        "HF_TOKEN": "REPLACE_WITH_HUGGINGFACE_TOKEN" # huggingface token to access gated models, e.g. llama 2
    },
)

In [ ]:
# define a data input dictonary with our uploaded s3 uris
data = {'training': training_input_path}

# starting the train job with our uploaded datasets as input
huggingface_estimator.fit(data, wait=True)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: gemma-9b-it-text-to-sql-2024-07-17-13-26-11-429


2024-07-17 13:26:15 Starting - Starting the training job...
2024-07-17 13:26:33 Starting - Preparing the instances for training...
2024-07-17 13:27:09 Downloading - Downloading input data...
2024-07-17 13:27:24 Downloading - Downloading the training image...............
2024-07-17 13:30:26 Training - Training image download completed. Training in progress....bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2024-07-17 13:31:00,660 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2024-07-17 13:31:00,678 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-07-17 13:31:00,689 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2024-07-17 13:31:00,690 sagemaker_pytorch_container.training INFO     Invoking user training script.
2024-07-17 13:31:02,173 sagemaker-training-toolkit INFO     Installing dependencies f